In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir("..")

In [16]:
import numpy as np
import torch
import random
import pickle
import yaml
# from omegaconf import OmegaConf
import glob
import types
# from q_probe import sweep_utils, rl_networks

from tqdm import tqdm
# from matplotlib import pyplot as plt
# import seaborn as sns

# import pyvene as pv
from transformers import AutoTokenizer, AutoModelForCausalLM

from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import TerminalFormatter
# from bigcode_eval import tasks
import llama

from pprint import pprint
def ppprint(code):
    print(highlight(code, PythonLexer(), TerminalFormatter()))

In [17]:
model_name = "baffo32/decapoda-research-llama-7B-hf"
model = llama.LlamaForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")
tokenizer = llama.LlamaTokenizer.from_pretrained(model_name)
print(model)

# https://github.com/huggingface/transformers/blob/main/src/transformers/models/llama/modeling_llama.py#L658

/n/holylfs06/LABS/kempner_undergrads/Users/jujipotle/envs/iti_h100/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:519: UserWarning: `pad_token_id` should be positive but got -1. This will cause errors when batch generating, if there is padding. Please set `pad_token_id` explicitly by `model.generation_config.pad_token_id=PAD_TOKEN_ID` to avoid errors in generation, and ensure your `input_ids` input does not have negative values.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 33/33 [00:19<00:00,  1.66it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=31999)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
          (att_out): Identity()
          (value_out): Identity()
          (head_out): Identity()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSN


normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [15]:
model_name = "meta-llama/Meta-Llama-3-8B"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(model)

# https://github.com/huggingface/transformers/blob/main/src/transformers/models/llama/modeling_llama.py#L658

Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.64s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head)

In [5]:
def wrapper(intervener):
    def wrapped(*args, **kwargs):
        return intervener(*args, **kwargs)
    return wrapped

## Check the baseline

In [6]:
q = "What to do in San Francisco?"
prompt = tokenizer(q, return_tensors="pt").to("cuda")

# run the uninterned forward pass
outputs = model.generate(
    prompt["input_ids"],
    max_new_tokens=256, 
    do_sample=False, 
)
print("Unintervened:")
pprint(tokenizer.decode(outputs[0], skip_special_tokens=True))  # includes the prompt

/n/holylfs05/LABS/pfister_lab/Lab/coxfs01/pfister_lab2/Lab/like/miniconda3/envs/q-probe/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/n/holylfs05/LABS/pfister_lab/Lab/coxfs01/pfister_lab2/Lab/like/miniconda3/envs/q-probe/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end ge

Unintervened:
('What to do in San Francisco? 10 things to do in San Francisco\n'
 'San Francisco is a city that is full of life and energy. There are so many '
 'things to do in San Francisco that it can be hard to know where to start. '
 'Here are 10 things to do in San Francisco that will help you make the most '
 'of your time in this amazing city.\n'
 '1. Visit the Golden Gate Bridge\n'
 'The Golden Gate Bridge is one of the most iconic landmarks in San Francisco. '
 'It is a must-see for any visitor to the city. The bridge is 1.7 miles long '
 'and 90 feet wide. It is made of steel and concrete and was completed in '
 '1937. The bridge is a popular spot for tourists to take photos and enjoy the '
 'views of the city and the bay.\n'
 '2. Take a cable car ride\n'
 'The cable cars are a San Francisco icon. They are a great way to see the '
 'city and get around. The cable cars run on tracks that are laid on the '
 'street. The cars are pulled by a cable that runs through the track. T

## Create the feature for a certain goal state

In [7]:
layers = range(16, 32, 4)

In [8]:
q = "The Tenderloin."
prompt = tokenizer(q, return_tensors="pt").to("cuda")

class Collector():
    def __init__(self):
        self.collections = []
    def __call__(self, b, s): 
        print(b.shape)
        self.collections.append(b)
        return b

config = []
collotors = []
for layer in layers:
    collotor = Collector()
    collotors.append(collotor)
    config.append({
        "component": f"model.layers[{layer}].self_attn.o_proj.input",
        "intervention": wrapper(collotor), 
        })

pv_model = pv.IntervenableModel(config, model=model)

_, intervened_outputs = pv_model(prompt)
goals = [collotor.collections[0][0, 0] for collotor in collotors]

torch.Size([1, 6, 4096])
torch.Size([1, 6, 4096])
torch.Size([1, 6, 4096])
torch.Size([1, 6, 4096])


In [13]:
class Intervener():
    def __init__(self, goal):
        self.goal = goal
        self.norm_intervene = 1.
        self.alpha = 0.5
    def __call__(self, b, s): 
        distance_to_goal = torch.norm(b[:, -1] - self.goal, dim=-1).item()
        direction_to_goal = b[:, -1] - self.goal
        # direction_to_goal = direction_to_goal / torch.norm(direction_to_goal, dim=-1, keepdim=True)
        b[:, -1] = b[:, -1] - direction_to_goal * self.alpha
        end_distance_to_goal = torch.norm(b[:, -1] - self.goal, dim=-1).item()
        # print(distance_to_goal, "--->", end_distance_to_goal)
        return b

config = []
interveners = []
for layer, goal in zip(layers, goals):
    intervener = Intervener(goal=goal)
    interveners.append(intervener)
    config.append({
        "component": f"model.layers[{layer}].mlp.output",
        "intervention": wrapper(intervener), 
        })


pv_model = pv.IntervenableModel(config, model=model)

q = "What to do in San Francisco?"
prompt = tokenizer(q, return_tensors="pt").to("cuda")

# run the intervened forward pass
_, intervened_outputs = pv_model.generate(
    prompt,
    max_new_tokens=256, 
    do_sample=False, 
)
print("Intervened: ")
pprint(tokenizer.decode(intervened_outputs[0], skip_special_tokens=True))  # includes the prompt


/n/holylfs05/LABS/pfister_lab/Lab/coxfs01/pfister_lab2/Lab/like/miniconda3/envs/q-probe/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/n/holylfs05/LABS/pfister_lab/Lab/coxfs01/pfister_lab2/Lab/like/miniconda3/envs/q-probe/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Intervened: 
('What to do in San Francisco? 10 things to do in San Francisco\n'
 'San Francisco is a city that is full of life and culture. It is a city that '
 'is always changing and evolving, and there are always new things to see and '
 'do. Here are 10 things to do in San Francisco:\n'
 '1. Visit the Golden Gate Bridge: The Golden Gate Bridge is one of the most '
 'iconic landmarks in the world, and it is definitely worth a visit. You can '
 'walk or bike across the bridge, or you can take a ferry across the bay.\n'
 '2. Visit Fisherman’s Wharf: Fisherman’s Wharf is a great place to visit, '
 'especially if you are looking for seafood. There are also a lot of shops and '
 'restaurants in the area, and it is a great place to people-watch.\n'
 '3. Visit Chinatown: Chinatown is one of the largest Chinatowns in the world, '
 'and it is definitely worth a visit. There are a lot of shops and restaurants '
 'in the area, and it is a great place to experience Chinese culture.\n'
 '4. Visi

In [16]:
type(pv_model)

pyvene.models.intervenable_base.IntervenableModel